# Обработка датасета

- rename_on_order_in_folder - Переименовать изображения и метки по порядку


- ImageLabelViewer - класс для просмотра изображений (run_viewer() - запуск)


# Библиотеки

In [7]:
import os
import cv2

from Module import rename_on_order_in_folder
# from Module import ImageLabelViewer, DatasetCreate

# Константы

In [8]:
FOLDER_WINTER_PATH = "./DataSet/MyData/WinterAll/"
FOLDER_SUMMER_PATH = "./DataSet/MyData/SummerAll/"

FOLDER_TRAIN_PATH = "./DataSet/NewDataAll/Train/"
FOLDER_TRAIN_PATH = "./DataSet/SplitDataScale/Train2/"
FOLDER_VALID_PATH = "./DataSet/NewDataAll/Valid/"
FOLDER_TEST_PATH = "./DataSet/NewDataAll/Test/"

Переименовать файлы в папке поочереди

In [26]:


import os

# Переименовать изображения и метки по порядку
def rename_on_order_in_folder(path, newName = "img_"):
    '''
        Переименовать все файлы в папке попорядку

        - path - путь
        - newName - имя для переименования
    '''
    images_folder = f"{path}images"
    labels_folder = f"{path}labels"

    images_files = sorted(os.listdir(images_folder))
    labels_files = sorted(os.listdir(labels_folder))

    # if len(images_files) != len(labels_files):
    #     print("Количество файлов в папках не совпадает!")
    # else:
        # Переименовываем файлы в обоих папках
    for i, (img_file, lbl_file) in enumerate(zip(images_files, labels_files), start=1):
        # Новый формат имени
        new_name = f"{newName}{i}"
        
        # Определяем расширения файлов
        img_ext = os.path.splitext(img_file)[1]
        lbl_ext = os.path.splitext(lbl_file)[1]

        # Полные пути к старым и новым файлам
        old_img_path = os.path.join(images_folder, img_file)
        new_img_path = os.path.join(images_folder, f"{new_name}{img_ext}")
        
        old_lbl_path = os.path.join(labels_folder, lbl_file)
        new_lbl_path = os.path.join(labels_folder, f"{new_name}{lbl_ext}")

        # Переименовываем файлы
        os.rename(old_img_path, new_img_path)
        os.rename(old_lbl_path, new_lbl_path)

    print("Файлы успешно переименованы!")

Расформировать данные train valid test

In [32]:
rename_on_order_in_folder(FOLDER_WINTER_PATH, newName="winter_")

Файлы успешно переименованы!


In [39]:
import os
import random
import shutil
from tqdm import tqdm

class DatasetCreate:
    '''
        Формирования дата сета для обучения
    '''

    def __init__(self, path_winter, path_summer, path_train, path_valid, path_test):
        self.path_winter = path_winter
        self.path_summer = path_summer
        self.path_train = path_train
        self.path_valid = path_valid
        self.path_test = path_test

        self.data_index = 0

    def get_split_data_for_train(self, data, train = 7, valid = 2, test = 1):
        '''
            Делим на 3 части
        '''
        
        total_length = len(data)
        part1_length = (total_length * 7) // 10 
        part2_length = (total_length * 2) // 10 

        train = data[:part1_length]
        valid = data[part1_length:part1_length + part2_length]
        test = data[part1_length + part2_length:]

        return train, valid, test

    def reset_directory(self, directory):
        '''
            Отчищаем дерикторию
        '''

        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)

    def copy_data(self, data, path_from, path_to, isClear = False, text = ""):
        '''
            Копируем дата данные в нужную папку
        '''

        if isClear:
            self.reset_directory(f'{path_to}/images')
            self.reset_directory(f'{path_to}/labels')

        for img in tqdm(data, desc=f"Копирование файлов {text}", unit="файл"):
            src_img = os.path.join(f'{path_from}/images', img)
            dst_img = os.path.join(f'{path_to}/images', f'{self.data_index}.jpg')
            shutil.copy(src_img, dst_img)

            # Находим соответствующую метку
            label_name = img.replace('.jpg', '.txt')
            label_name = label_name.replace('.JPG', '.txt')
            src_label = os.path.join(f'{path_from}/labels', label_name)
            dst_label = os.path.join(f'{path_to}/labels', f'{self.data_index}.txt')
            # print(src_label, dst_label)
            shutil.copy(src_label, dst_label)

            self.data_index += 1

    def create_data_for_dataset(self):
        '''
            Формирвоание данных для обучения
        '''

        files_winter_images = os.listdir(f'{self.path_winter}images')
        files_summer_images = os.listdir(f'{self.path_summer}images')

        # перемешали 
        random.shuffle(files_winter_images)
        random.shuffle(files_summer_images)

        train_w, valid_w, test_w = self.get_split_data_for_train(files_winter_images)
        train_s, valid_s, test_s = self.get_split_data_for_train(files_summer_images)

        self.data_index = 0
        self.copy_data(train_w, self.path_winter, self.path_train, True, text="train зимняя")
        self.copy_data(train_s, self.path_summer, self.path_train, False, text="train зимняя")

        self.data_index = 0
        self.copy_data(valid_w, self.path_winter, self.path_valid, True, text="valid зимняя")
        self.copy_data(valid_s, self.path_summer, self.path_valid, False, text="valid зимняя")

        self.data_index = 0
        self.copy_data(test_w, self.path_winter, self.path_test, True, text="test зимняя")
        self.copy_data(test_s, self.path_summer, self.path_test, False, text="test зимняя")

In [40]:
new_data = DatasetCreate(
    FOLDER_WINTER_PATH, 
    FOLDER_SUMMER_PATH, 
    FOLDER_TRAIN_PATH, 
    FOLDER_VALID_PATH, 
    FOLDER_TEST_PATH)

new_data.create_data_for_dataset()


Копирование файлов test зимняя: 100%|██████████| 247/247 [00:05<00:00, 41.69файл/s]


Просмотр картинко с рамками в папке

In [16]:
import os
import cv2

class ImageLabelViewer:
    '''
        Просмотр изображений с масской

        - path - путь к папке
        - display_size - размер изображения
        -------------------------------------------
        - run_viewer - запуск
    '''

    def __init__(self, path, display_size=(1280, 880)):
        self.images_folder = os.path.join(path, "images")
        self.labels_folder = os.path.join(path, "labels")
        
        # self.images_files = sorted(os.listdir(self.images_folder))
        # self.labels_files = sorted(os.listdir(self.labels_folder))

        self.images_files = os.listdir(self.images_folder)
        self.labels_files = os.listdir(self.labels_folder)
        
        # if len(self.images_files) != len(self.labels_files):
        #     raise ValueError("Количество файлов в папках не совпадает!")
        
        self.current_index = 0
        self.display_size = display_size

    def _draw_bounding_boxes(self, image, label_file):
        h, w = image.shape[:2]

        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                # class_id = int(parts[0])
                x_center, y_center, box_width, box_height = map(float, parts[1:])

                # Переводим центр и размеры в абсолютные координаты
                x_min = int((x_center - box_width / 2) * w)
                y_min = int((y_center - box_height / 2) * h)
                x_max = int((x_center + box_width / 2) * w)
                y_max = int((y_center + box_height / 2) * h)

                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
        return image

    def _show_image_with_boxes(self):
        # Получаем текущее изображение и соответствующий файл меток
        img_file = os.path.join(self.images_folder, self.images_files[self.current_index])
        lbl_file = os.path.join(self.labels_folder, self.labels_files[self.current_index])
        image = cv2.imread(img_file)
        
        cv2.putText(image, f"image-{self.current_index}", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4)
        cv2.putText(image, "A-back, d-next, esc-exit", (10,120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 4)

        image_with_boxes = self._draw_bounding_boxes(image, lbl_file)
        image_with_boxes = cv2.resize(image_with_boxes, self.display_size)
        cv2.imshow(f"Image", image_with_boxes)

    def _next_image(self):
        # Переход к следующему изображению
        self.current_index = (self.current_index + 1) % len(self.images_files)
        self._show_image_with_boxes()
    
    def _prev_image(self):
        # Переход к предыдущему изображению
        self.current_index = (self.current_index - 1) % len(self.images_files)
        self._show_image_with_boxes()

    def run_viewer(self):
        # Показать первое изображение
        self._show_image_with_boxes()

        while True:
            # Ожидаем нажатие клавиши
            key = cv2.waitKey(0)

            # Обработка нажатия клавиш
            if key == ord('d'):  # 'n' - следующее изображение
                self._next_image()
            elif key == ord('a'):  # 'p' - предыдущее изображение
                self._prev_image()
            elif key == 27:  # ESC - выход
                break

        # Закрываем окно
        cv2.destroyAllWindows()

In [24]:
FOLDER_TRAIN_PATH = "./DataSet/SplitData/Valid/"

# использование класса
viewer = ImageLabelViewer(FOLDER_TRAIN_PATH)
viewer.run_viewer()